In [1]:
from bs4 import BeautifulSoup

import datetime
import requests

import pandas as pd
import numpy as np

import pymongo
import json

## 1. Philadelphia Happy Hour Special Data

### 1a. Extract happy hour special data from https://philly.thedrinknation.com/specials#

* Day of week
* Name of restaurant
* Neighborhood
* Address
* Website
* Special "type"
* Special time
* Special Description

### 1b. Perform the following transformations
* Remove all newline characters from strings generated by scrape output using python's .replace() method
* Remove leading and trailing whitespace using python's .strip() method

In [2]:
url = 'https://philly.thedrinknation.com/specials/Wednesday'

In [3]:
#Retrieve page with the requests module
response = requests.get(url)

In [4]:
# BeautifulSoup object, parse
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Print
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
 <head>
  <meta content="tgns43adAzj5zO2k-Ak61sQWjVmObzpXSizxoPcA2X4" name="google-site-verification"/>
  <link href="https://philly.thedrinknation.com/mobile" media="only screen and (max-width: 640px)" rel="alternate"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Ready for happy hour? Philadelphia bars and clubs are featured on Drink Philly has to offer." name="description"/>
  <meta content="happy hour philadelphia" name="keywords"/>
  <title>
   Happy Hour Philadelphia | Specials | Drink Philly - Drink Philly
  </title>
  <link href="//philly.thedrinknation.com/favicon.ico" rel="shortcut icon">
   <!--link rel="stylesheet" href="https://philly.thedrinknation.com/css/reset.css" /-->
   <!--link rel="stylesheet" href="https://philly.t

In [6]:
# Since page is structured to show specials in separate links by day of week, create 'helper' dictionary to iterate through each day
helper_dict = {1:'Sunday',
               2:'Monday',
               3:'Tuesday',
               4:'Wednesday',
               5:'Thursday',
               6:'Friday',
               7:'Saturday'}

# Create empty list to store results
results_list = []

for i in range(1,8):
    day_of_week = helper_dict[i]
    # Construct URL to scrape based off of day of week
    url = f'https://philly.thedrinknation.com/specials/{day_of_week}'
    # Update soup object with latest URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Grab all of the names, address, website, etc. using div tags with the 'barMeta' class
    results = soup.find_all('div', class_='barMeta')

    for result in results:
        # Name in restaurant - use text of first a tag
        restaurant_name = result.a.text.replace("\n","").strip()
        # Restaurant link - use a tag with class 'barLink'
        website = result.find('a', class_='barLink')['href'].replace("\n","").strip()
        # Neighborhood - span tag with class 'barNHood'
        neighborhood = result.find('span', class_='barNHood').text.replace("\n","").strip()
        # Address
        address = result.contents[2].replace("\n","").strip()
        # To grab special information, loop through the result's div tag with class 'todaysSpecial'
        todays_special = result.find('div', class_ = 'todaysSpecial')
        special_items = todays_special.find_all('div')
        # Create empty list to store all specials for individual restaurants
        specials_list = []
        for item in special_items:
            special_type = item.find('strong').text.replace("\n","").strip()
            special_time = item.find('span').text.replace("\n","").strip()
            special_description = item.contents[2].replace(":","").replace("\n","").strip()
            # Create dictionary of all specials offered by restaurant
            specials_dict = {'special_type':special_type,
                             'special_time':special_time,
                             'special_description':special_description
                            }
            specials_list.append(specials_dict)
        # Create dictionary to store individual result attributes
        results_dict  = {'day_of_week':day_of_week,
                         'restaurant_name':restaurant_name,
                         'website':website,
                         'neighborhood':neighborhood,
                         'address':address,
                         'specials':specials_list
                        }
        # Append results dictionary to results list 
        results_list.append(results_dict)
    
print(results_list)
    
    



[{'day_of_week': 'Sunday', 'restaurant_name': '1 Tippling Place', 'website': 'http://1tpl.com', 'neighborhood': 'Rittenhouse', 'address': '2006 Chestnut Street    (20th & Chestnut)', 'specials': [{'special_type': 'Late Night', 'special_time': '(        10:00 pm        -        12:00 am        )', 'special_description': '$1-2 off Select Cocktails; $5 Fish House Punch'}]}, {'day_of_week': 'Sunday', 'restaurant_name': '12 Steps Down', 'website': 'http://www.12stepsdown.com/', 'neighborhood': 'South Philly', 'address': '831 Christian Street    (9th & Christian)', 'specials': [{'special_type': 'Brunch', 'special_time': '(        11:00 am        -        3:00 pm        )', 'special_description': "$3.50 Bloody Mary's & Mimosas"}]}, {'day_of_week': 'Sunday', 'restaurant_name': '2nd Story Brewing', 'website': 'http://www.2ndstorybrewing.com', 'neighborhood': 'Old City', 'address': '117 Chestnut Street    (2nd & Chestnut)', 'specials': [{'special_type': 'All Day', 'special_time': '(        11:30

## 2. Extract neighborhood regions from CSV file using Pandas
### No additional transformations needed

In [7]:
# CSV file created from site https://www.philageohistory.org/rdic-images/common/help/PhilaRegions.cfm
source_file = 'philly_neighborhood_regions.csv'

In [8]:
region_df = pd.read_csv(source_file)
region_df.head()

Neighborhood       Region
0  Avenue of the Arts  Center City
1          Callowhill  Center City
2           Chinatown  Center City
3     Elfreth's Alley  Center City
4       Fitler Square  Center City

In [9]:
# Convert dataframe to json
region_json = region_df.to_json(orient='records')
region_json_loads = json.loads(region_json)
print(region_json_loads)

[{'Neighborhood': 'Avenue of the Arts', 'Region': 'Center City'}, {'Neighborhood': 'Callowhill', 'Region': 'Center City'}, {'Neighborhood': 'Chinatown', 'Region': 'Center City'}, {'Neighborhood': "Elfreth's Alley", 'Region': 'Center City'}, {'Neighborhood': 'Fitler Square', 'Region': 'Center City'}, {'Neighborhood': 'Franklintown', 'Region': 'Center City'}, {'Neighborhood': 'Logan Square', 'Region': 'Center City'}, {'Neighborhood': "Jewelers' Row", 'Region': 'Center City'}, {'Neighborhood': 'Market East', 'Region': 'Center City'}, {'Neighborhood': 'Old City', 'Region': 'Center City'}, {'Neighborhood': 'Museum District', 'Region': 'Center City'}, {'Neighborhood': 'Penn Center', 'Region': 'Center City'}, {'Neighborhood': "Penn's Landing", 'Region': 'Center City'}, {'Neighborhood': 'Rittenhouse Square', 'Region': 'Center City'}, {'Neighborhood': 'Society Hill', 'Region': 'Center City'}, {'Neighborhood': 'South Street', 'Region': 'Center City'}, {'Neighborhood': 'Washington Square West', '

## 3. Load to MongoDB 'philly_specials.db'
### 3a. Establish connection to MongoDB
### 3b. Load scraped data from happy hour specials site into 'restaurant_specials' collection
### 3c. Load neighborhood/region mapping file into 'neighborhoods' collection

In [10]:
# 3a. Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Select database and collection to use
db = client.philly_specials

# restaurant_specials collection
restaurant_specials = db.restaurant_specials
# neighborhoods collection
neighborhoods = db.neighborhoods

In [11]:
# 3b.) Load scraped data from happy hour specials site into 'restaurant_specials' collection
restaurant_specials.insert_many(results_list)
print('Data loaded!')

Data loaded!


In [12]:
# 3c.) Load neighborhood/region mapping file into 'neighborhoods' collection
neighborhoods.insert_many(region_json_loads)
print('Data loaded!')

Data loaded!


--------------------------------------------------------------------------------

# Writeup

### ETL Project: Philly Happy Hour Database

#### Extract

We decided to gather information for all restaurants in Philadelphia that offer specials, using the following data:

Web Scape via Beautiful Soup: https://philly.thedrinknation.com/specials/
This link provided our main source of restaurant data in Philadelphia and the information was embedded in the HTML code.
From the HTML, we extracted the following for each restaurant:
* Day of Week
* Name
* Website
* Neighborhood
* Address
* Special Type (Happy Hour, Brunch, etc.)
* Special’s time
* Special’s offer/description

We repeated this for each day of the week, as the website had separate webpages for each day.


CSV File: https://www.philageohistory.org/rdic-images/common/help/PhilaRegions.cfm
This link provided a list of all neighborhoods in Philadelphia.
* We attempted to scrape the tables in the html from the website directly via pandas, but the html table was not in the proper format for pandas to recognize.
* We created a csv file with this data, extracted it with pandas, and turned it into a json file.

#### Transform

For Restaurant Data:
We looped through each restaurant on the website and created a dictionary for each restaurant’s information in the following format:

{ “day_of_week”: day of week,
“restaurant_name”: restaurant name,
“website”: website,
“address”: address,
“special_type”: special type,
“specials”: specials_dict }

In order to parse through the types of specials that each restaurant had, we had to create a separate dictionary to store within the main dictionary under the “specials” key, with the following format:

{ “special_type”: special type, 
“special_time”: special time, 
“special_description”: special description }

This full structure was appended to a list and repeated for each day of the week.
We then removed any extra characters that existed for each string:

\n replaced with a blank space via .replace()
Whitespace removed via .split()

For Neighborhoods Data
We moved the data to a csv file and loaded it into a pandas dataframe. 
It was then put into a json file for storage with the following format:
{“neighborhood”:neighborhood,
”region”:region}

#### Load

Because of the way we structured the data (list of dictionaries), we decided to use a MongoDB Database. We believed it was favorable to use a non-relational database, since our dictionary also had an embedded dictionary, which would have been more challenging to transfer into a table for SQL.

The collections have same dictionary format as structured in the Transform section:

Database:
* philly_specials.db

Collections: 
* restaurant_specials
* neighborhoods


After creating the two collections, we were able to join them together using the $lookup aggregation function in MongoDB, such as the following:

db.philly_specials.aggregate([
  {
    $lookup:
      {
        from: ‘neighborhoods’,
        localField: ‘neighborhood’,
        foreignField: ‘Neighborhood’,
        as: ‘neighborhood_region’
      }
 }
])